In [10]:
from h2ogpte import H2OGPTE

In [16]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ',
)

# # Chat with LLM
# chat_session_id = client.create_chat_session()
# with client.connect(chat_session_id) as session:
#     # Simple Question for Document Collection
#     answer = session.query(
#         "Can you hallucinate?",
#     ).content
#     print(answer)

# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="PDF -> text -> summary",
)

import os

# Your existing code
file_path = "/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/Cutdown.xlsx"
with open(file_path, "rb") as f:
     songs = client.upload(os.path.basename(file_path), f)

file_path1 = "/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx"
with open(file_path1, "rb") as f:
     user = client.upload(os.path.basename(file_path1), f)

# with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx', 'rb') as f:
#     hist = client.upload(os.path.basename('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx'), f)

# with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx', 'rb') as f:
#     songs = client.upload(os.path.basename('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx'), f)

#with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/music.csv', 'rb') as f:
#    spot = client.upload(os.path.basename('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/music.csv'), f)

# # Converting the input into chunked text and embeddings...
client.ingest_uploads(collection_id, [songs,user])

Job(id='6f889932-3ff9-46f9-a972-1dd98a7b9fb6', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 28, 9, 9, 12, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 28, 9, 13, 29, tzinfo=TzInfo(UTC)), duration='4m17s', duration_seconds=257.0)

In [17]:
chat_session_id = client.create_chat_session(collection_id)
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="I am user 1. Reccomend me songs similar to my taste",
        system_prompt = 'Assume music and song to be the same word. When a question as for a similar music, recommend less than 5 music unless told otherwise. Find similar music based on genre and other factors such as danceability, loudness, speechiness and more. Just return the song name, unless stated otherwise.' ,
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on the information provided, you have listened to "All Along the Watchtower" by Jimi Hendrix, which is a classic rock song. Here are some similar songs that you might enjoy:

1. "Stairway to Heaven" by Led Zeppelin
2. "More Than a Feeling" by Boston
3. "Hotel California" by The Eagles
4. "Sweet Home Alabama" by Lynyrd Skynyrd
5. "Born to Be Wild" by Steppenwolf

These songs are all considered classics in the rock genre and have a similar sound and feel to "All Along the Watchtower."


Prompt designs

In [ ]:
print([x["base_model"] for x in client.get_llms()])

['mistralai/Mixtral-8x7B-Instruct-v0.1', 'h2oai/h2ogpt-4096-llama2-70b-chat', 'h2oai/h2ogpt-4096-llama2-13b-chat', 'h2oai/h2ogpt-32k-codellama-34b-instruct', 'HuggingFaceH4/zephyr-7b-beta', 'NousResearch/Nous-Capybara-34B', 'claude-2.1', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k-0613', 'gpt-35-turbo-1106', 'gpt-4-1106-preview', 'gemini-pro', 'mistral-small-latest', 'mistral-large-latest', 'mistral-medium']


In [ ]:
# Inspect and collect all text chunks
chunks = []
for chunk_id in range(1, 100):
    try:
        chunk = client.get_chunks(collection_id, [chunk_id])
        print(chunk, flush=True)
        chunks.append(chunk[0].text)
    except:
        break

print(f"Number of chunks: {len(chunks)}", flush=True)

Number of chunks: 0


In [ ]:
Prompt_template = """
Answer the questions based on only the following context:

{context}

---

Answer the questions based on the above context: {query}
"""
